In [ ]:
## Importing libraries ##

import pymc as pm
import corner
import warnings
import random
warnings.filterwarnings("ignore")

In [ ]:
## Importing libraries ##

import numpy as np
import arviz as az
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint
import scipy.stats as st
import pytensor
import pytensor.tensor as pt
from pytensor.compile.ops import as_op 
import numbers
import seaborn as sns

from pymc.ode import DifferentialEquation
print(f"Running on PyMC v{pm.__version__}")

RANDOM_SEED = 60
rng = np.random.default_rng(RANDOM_SEED)

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"

In [ ]:
## Reading stress buildup data for DOWSIL TC-5550 ##

df= pd.read_excel("DOWTC5550_030323.xlsx",header = None, names=['shear rate','t','shear stress'],sheet_name=2,skiprows=range(155,298))
Data = df.to_numpy()

times = Data[:,1]
shear_stress_0p08s = Data[:,2].reshape(-1,1)

shear_rate_ode = 0.08
print(np.shape(times))
print(np.shape(shear_stress_0p08s))

# Defining the true (or PINN) solution to unknown model parameters of NEVP constitutive model #
G_true = 8972.0
eta_s_true = 363.0
sigma_y_true = 6.9
k_true = 214.2
n_true = 0.85

In [ ]:
## Data processing ##

t = times
# IC at different shear rates from experimental data #
y0_all = [9.141,10.003,11.966,12.478,14.668]
# Shear rates at which experiments were performed #
shear_rate_all = [0.06,0.07,0.08,0.09,0.1]
# Training data corresponds to 0.06/s, 0.07/s and 0.08/s shear-rates #
y0_train = [9.141,10.003,11.966]
shear_rate_train = [0.06,0.07,0.08]
shear_stress_global = np.zeros((155,5))

# Data processing to read all shear stress and then defining training data for 0.07/s, 0.08/s and 0.09/s #
for itr in range(0,5):
  # if itr==2:
  #   continue
  df= pd.read_excel("DOW1_030323.xlsx",header = None, names=['shear rate','t','shear stress'],sheet_name=itr,skiprows=range(155,298))
  Data = df.to_numpy()
  shear_stress_local = Data[:,2]
  print(np.shape(shear_stress_local))
  shear_stress_global[:,itr] = shear_stress_local

shear_stress_training = np.zeros(465,)
shear_stress_training[0:155] = shear_stress_global[:,0]
shear_stress_training[155:310] = shear_stress_global[:,1]
shear_stress_training[310:465] = shear_stress_global[:,2]
print(shear_stress_training) # contains data for 0.06/s, 0.07/s and 0.08/s
print(np.shape(shear_stress_training)) 

# Appending time to create times_1d #
times_1d = np.append(times,times)
times_1d = np.append(times_1d,times)
times_1d = times_1d.reshape(-1)
print(np.shape(times_1d))

# Shear rate index #
shear_rate_0p06_idx = 0*np.ones(155)
shear_rate_0p07_idx = 1*np.ones(155)
shear_rate_0p08_idx = 2*np.ones(155)
shear_rate_idx = np.append(shear_rate_0p06_idx,shear_rate_0p07_idx)
shear_rate_idx = np.append(shear_rate_idx,shear_rate_0p08_idx)
shear_rate_idx = shear_rate_idx.reshape(-1)
shear_rate_idx = np.asarray(shear_rate_idx,dtype='int')
print(np.shape(shear_rate_idx))

# shear rates under consideration #
shear_rate_0p06 = 0.06*np.ones(155)
shear_rate_0p07 = 0.07*np.ones(155)
shear_rate_0p08 = 0.08*np.ones(155)
shear_rate_1d = np.append(shear_rate_0p06,shear_rate_0p07) 
shear_rate_1d = np.append(shear_rate_1d,shear_rate_0p08)
shear_rate_1d = shear_rate_1d.reshape(-1)
print(np.shape(shear_rate_1d))

# ICs under consideration #
stress_ic_0p06 = y0_all[0]*np.ones(155)
stress_ic_0p07 = y0_all[1]*np.ones(155)
stress_ic_0p08 = y0_all[2]*np.ones(155)
stress_ic_1d = np.append(stress_ic_0p06,stress_ic_0p07) 
stress_ic_1d = np.append(stress_ic_1d,stress_ic_0p08)
stress_ic_1d = stress_ic_1d.reshape(-1)
print(np.shape(stress_ic_1d))

In [ ]:
## Defining min and max values to infer all model parameters between 0-1 ##
G_min = 5000
G_max = 9000
eta_s_min = 100 #100
eta_s_max = 400 #400
sigma_y_min = 0
sigma_y_max = 10
k_min = 200
k_max = 300
n_min = 0.5
n_max = 1

## Analytical solution of the NEVP constitutive model ##
def shear_stress_analytical(G,eta_s,yield_stress,k,n,shear_rate_ode,stress_ic,time_ode):
    # input arguments of model parameters range is 0-1
    # dimensionalizing the model parameters
    G_d = G*(G_max-G_min) + G_min
    #print(G_d)
    eta_s_d = eta_s*(eta_s_max-eta_s_min) + eta_s_min
    #print(eta_s_d)
    yield_stress_d = yield_stress*(sigma_y_max-sigma_y_min) + sigma_y_min
    #print(yield_stress_d)
    k_d = k*(k_max-k_min) + k_min
    #print(k_d)
    n_d = n*(n_max-n_min) + n_min
    #print(n_d)
    
    shear_stress_an = (yield_stress_d + k_d*shear_rate_ode**n_d)*(1 - (1-(stress_ic/(yield_stress_d + k_d*shear_rate_ode**n_d))) * np.exp(-G_d*shear_rate_ode*(time_ode-0.031)/(yield_stress_d+ k_d*shear_rate_ode**n_d + eta_s_d*shear_rate_ode)))
    
    return shear_stress_an

In [ ]:
## Plotting the stress buildup data of DOWSIL TC-5550 ##

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"
sns.set_style(None)

def dy_dt(y,t,G,eta_s,yield_stress,k,n,shear_rate_ode):
  y1 = y
  dydt = (G*shear_rate_ode/(yield_stress + k*shear_rate_ode**n + eta_s*shear_rate_ode))*(-y1 + yield_stress + k*shear_rate_ode**n)
  return dydt


for itr in range(len(shear_rate_all)+1):
  if 0<=itr<=2:  
      plt.plot(times,shear_stress_global[:,itr],'og',alpha=1.0)
  elif 3<=itr<5:
      plt.plot(times,shear_stress_global[:,itr],'oy',alpha=1.0)
  elif itr == 5:
      plt.plot(np.NaN,np.NaN,'og',alpha=1.0,label='Observational data')
      plt.plot(np.NaN,np.NaN,'oy',alpha=1.0,label='Validation data')
     
  plt.arrow(0.5,20,0.5,20,head_width=0.2,head_length=2,fc="k",ec="k",width=0.01)
    
plt.xlabel(r'$t$ (s)')
plt.ylabel(r'$\sigma_{\mathrm{exp}}$ (Pa)')
plt.xscale('log')
plt.legend()
plt.savefig('NEVP_Training_Test_data.png',format='png',bbox_inches='tight'),
plt.savefig('NEVP_Training_Test_data.eps',format='eps',bbox_inches='tight')
plt.show()

In [ ]:
## To define dimensionality of unknown model parameters in the model definition below ##
coords = {"shear_rate_coord": shear_rate_train}

In [ ]:
## Defining the UQ model ##
with pm.Model(coords=coords) as model:
    time_data = pm.ConstantData("time_data", times_1d, dims="obs_id")
    print(pm.draw(time_data).size)
    
    # Hyper-priors: inference space between 0 and 1 #
    #mu_g = pm.LogNormal("mu_g",mu=0.0,sigma=1.0) #0.0 0.25
    mu_g = pm.Beta("mu_g",alpha=2.0,beta=2.0) 
    sigma_g = pm.HalfNormal("sigma_g",sigma=0.05) #0.01
    
    #mu_eta_s = pm.LogNormal("mu_eta_s",mu=0.0,sigma=1.0) 
    mu_eta_s = pm.Beta("mu_eta_s",alpha=2.0,beta=2.0)
    sigma_eta_s = pm.HalfNormal("sigma_eta_s",sigma=0.05)
    
    #mu_yield_stress = pm.HalfNormal("mu_yield_stress",sigma=0.1) 
    mu_yield_stress = pm.Beta("mu_yield_stress",alpha=2.0,beta=2.0)
    sigma_yield_stress = pm.HalfNormal("sigma_yield_stress",sigma=0.05)
    
    #mu_k = pm.HalfNormal("mu_k",sigma=0.5) #0.5
    mu_k = pm.Beta("mu_k",alpha=2.0,beta=2.0)
    sigma_k = pm.HalfNormal("sigma_k",sigma=0.05) # 0.05 increasing this sigma value leads to unphysical stress values - 1e32 or so
    
    mu_n = pm.Beta("mu_n",alpha=2.0,beta=2.0) # Beta because n = [0,1] 
    sigma_n = pm.HalfNormal("sigma_n",sigma=0.05)
    
    # Priors: normalized and dimensional unknown model parameters using non-centered paramterization #
    z_G = pm.Normal("z_G",mu=0,sigma=1,dims="shear_rate_coord")
    G = pm.Deterministic("G", mu_g+sigma_g*z_G, dims="shear_rate_coord")
    G_dd = pm.Deterministic("G_dd", G*(G_max-G_min)+G_min, dims="shear_rate_coord")
    
    z_eta_s = pm.Normal("z_eta_s",mu=0,sigma=1,dims="shear_rate_coord")
    eta_s = pm.Deterministic("eta_s", mu_eta_s + sigma_eta_s*z_eta_s, dims="shear_rate_coord")
    eta_s_dd = pm.Deterministic("eta_s_dd", eta_s*(eta_s_max-eta_s_min)+eta_s_min, dims="shear_rate_coord")
    
    z_yield_stress = pm.Normal("z_yield_stress",mu=0,sigma=1,dims="shear_rate_coord")
    yield_stress = pm.Deterministic("yield_stress", mu_yield_stress + sigma_yield_stress*z_yield_stress , dims="shear_rate_coord")
    yield_stress_dd = pm.Deterministic("yield_stress_dd", yield_stress*(sigma_y_max-sigma_y_min)+sigma_y_min, dims="shear_rate_coord")

    z_k = pm.Normal("z_k",mu=0,sigma=1,dims="shear_rate_coord")
    k = pm.Deterministic("k", mu_k + sigma_k*z_k, dims="shear_rate_coord")
    k_dd = pm.Deterministic("k_dd", k*(k_max-k_min)+k_min, dims="shear_rate_coord")

    z_n = pm.Normal("z_n",mu=0,sigma=1,dims="shear_rate_coord")
    n = pm.Deterministic("n", mu_n + sigma_n*z_n, dims="shear_rate_coord")
    n_dd = pm.Deterministic("n_dd", n*(n_max-n_min)+n_min, dims="shear_rate_coord")

    # Experiment measurement uncertainty #
    sigma = pm.Exponential("sigma", 2.0) # rate parameter = 5.0  2.0
    
    # Likelihood mean based on analytical solution #
    shear_stress_mean = (yield_stress_dd[shear_rate_idx] + k_dd[shear_rate_idx]*shear_rate_1d**n_dd[shear_rate_idx])* \
    (1 - (1-(stress_ic_1d/(yield_stress_dd[shear_rate_idx] + k_dd[shear_rate_idx]*shear_rate_1d**n_dd[shear_rate_idx]))) * \
    np.exp(-G_dd[shear_rate_idx]*shear_rate_1d*(time_data-0.031)/(yield_stress_dd[shear_rate_idx]+ k_dd[shear_rate_idx]*shear_rate_1d**n_dd[shear_rate_idx] + eta_s_dd[shear_rate_idx]*shear_rate_1d)))
    
    print(pm.draw(shear_stress_mean).shape)

    # Likelihood #
    pm.Laplace("Y_obs", mu=shear_stress_mean, b=sigma, observed=shear_stress_training, dims = "obs_id") 
    idata = pm.sample_prior_predictive(samples=10000, random_seed=rng)

print(idata)

In [ ]:
pm.model_to_graphviz(model=model)

In [ ]:
# Plotting priors to understand data-construction within imdata_df datadrame #

imdata_df = az.extract(idata, group = "prior", num_samples=2).to_dataframe()
print(imdata_df)

cols = ["mu_g", "mu_eta_s", "mu_yield_stress", "mu_k", "mu_n"]
row = imdata_df.iloc[0, :][cols].values
print(row)
cols = ["sigma_g", "sigma_eta_s", "sigma_yield_stress", "sigma_k", "sigma_n"]
row = imdata_df.iloc[0, :][cols].values
print(row)
cols = ["z_G", "z_eta_s", "z_yield_stress", "z_k", "z_n"]
row = imdata_df.iloc[0, :][cols].values
print(row)
cols = ["G", "eta_s", "yield_stress", "k", "n"]
row = imdata_df.iloc[0, :][cols].values
print(row)

In [ ]:
## Plotting priors predictive - before sampling ##

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"

num_samples = 100
imdata_prior = az.extract(idata, group = "prior", num_samples=num_samples).to_dataframe()
cols = ["G", "eta_s", "yield_stress", "k", "n"]

# arg: 0 (0.06/s); 1 (0.07/s); 2 (0.08/s); 3 (0.09/s); 4 (0.1/s);
arg = 0
shear_rate_ode = shear_rate_all[arg]
y0 = y0_all[arg]
# Using dimensional time and dimensional IC to get dimensional shear stress
# stress_ode = odeint(dy_dt,y0,t,args=(G_true,eta_s_true,sigma_y_true,k_true,n_true,shear_rate_ode))
stress_analytical_pinn = shear_stress_analytical((G_true-G_min)/(G_max-G_min),(eta_s_true-eta_s_min)/(eta_s_max-eta_s_min)\
                                                 ,sigma_y_true/sigma_y_max,(k_true-k_min)/(k_max-k_min),(n_true-n_min)/(n_max-n_min),shear_rate_ode*np.ones(155),y0,times)
shear_stress_expt = shear_stress_global[:,arg]

plt.plot(times,shear_stress_expt,'og', label='Experiment data')
plt.plot(times,stress_analytical_pinn,'-k', label='PINN analytical solution',linewidth=3)
#plt.plot(times,stress_ode,'-b', label='PINN ODE solution')
for row_idx in range(0,num_samples*3,3):
  row = imdata_prior.iloc[row_idx, :][cols].values
  theta = row
  # Using dimensional time as input to get dimensional shear_stress as output
  shear_stress_an = shear_stress_analytical(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode,y0,times)#,shear_rate_1d,times_1d)
  #stress_odes = odeint(dy_dt,theta[5],t,args=(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode))
  plt.plot(times,shear_stress_an,'-b',alpha=0.1)
plt.plot(np.NaN,np.NaN,'-g',alpha=0.1,label='Prior Samples')

plt.title(r'$0.06$/s')
plt.xlabel(r'$t$ (s)',fontsize=16)
plt.ylabel(r'$\sigma$ (Pa)',fontsize=16)
plt.xscale('log')
plt.legend()
#plt.savefig('0p06_prior_legend.png',format='png')
plt.show()

arg = 1
shear_rate_ode = shear_rate_all[arg]
y0 = y0_all[arg]
# Using dimensional time and dimensional IC to get dimensional shear stress
#stress_ode = odeint(dy_dt,y0,t,args=(G_true,eta_s_true,sigma_y_true,k_true,n_true,shear_rate_ode))
stress_analytical_pinn = shear_stress_analytical((G_true-G_min)/(G_max-G_min),(eta_s_true-eta_s_min)/(eta_s_max-eta_s_min)\
                                                 ,sigma_y_true/sigma_y_max,(k_true-k_min)/(k_max-k_min),(n_true-n_min)/(n_max-n_min),shear_rate_ode*np.ones(155),y0,times)
shear_stress_expt = shear_stress_global[:,arg]

plt.plot(times,shear_stress_expt,'og', label='Experiment data')
plt.plot(times,stress_analytical_pinn,'-k', label='PINN analytical solution',linewidth=3)
#plt.plot(times,stress_ode,'-b', label='PINN ODE solution')
for row_idx in range(1,num_samples*3,3):
  row = imdata_prior.iloc[row_idx, :][cols].values
  theta = row
  # Using dimensional time as input to get dimensional shear_stress as output
  shear_stress_an = shear_stress_analytical(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode,y0,times) #,shear_rate_1d,times_1d)
  if np.sum(shear_stress_an) > 1e6:
      print(shear_stress_an)
    #stress_odes = odeint(dy_dt,theta[5],t,args=(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode))
  plt.plot(times,shear_stress_an,'-b',alpha=0.1)
plt.plot(np.NaN,np.NaN,'-g',alpha=0.1,label='Prior Samples')

plt.title(r'$0.07$/s')
plt.xlabel(r'$t$ (s)')
plt.ylabel(r'$\sigma$ (Pa)')
plt.xscale('log')
#plt.legend()
#plt.savefig('0p07_prior.png',format='png')
plt.show()

arg = 2
shear_rate_ode = shear_rate_all[arg]
y0 = y0_all[arg]
# Using dimensional time and dimensional IC to get dimensional shear stress
#stress_ode = odeint(dy_dt,y0,t,args=(G_true,eta_s_true,sigma_y_true,k_true,n_true,shear_rate_ode))
stress_analytical_pinn = shear_stress_analytical((G_true-G_min)/(G_max-G_min),(eta_s_true-eta_s_min)/(eta_s_max-eta_s_min)\
                                                 ,sigma_y_true/sigma_y_max,(k_true-k_min)/(k_max-k_min),(n_true-n_min)/(n_max-n_min),shear_rate_ode*np.ones(155),y0,times)
shear_stress_expt = shear_stress_global[:,arg]

plt.plot(times,shear_stress_expt,'og', label='Experiment data')
plt.plot(times,stress_analytical_pinn,'-k', label='PINN analytical solution',linewidth=3)
#plt.plot(times,stress_ode,'-b', label='PINN ODE solution')
for row_idx in range(2,num_samples*3,3):
  row = imdata_prior.iloc[row_idx, :][cols].values
  theta = row
  # Using dimensional time as input to get dimensional shear_stress as output
  shear_stress_an = shear_stress_analytical(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode,y0,times) #,shear_rate_1d,times_1d)
  if np.sum(shear_stress_an) > 1e6:
      print(shear_stress_an)
    #stress_odes = odeint(dy_dt,theta[5],t,args=(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode))
  plt.plot(times,shear_stress_an,'-b',alpha=0.1)
plt.plot(np.NaN,np.NaN,'-g',alpha=0.1,label='Prior Samples')

plt.title(r'$0.08$/s')
plt.xlabel(r'$t$ (s)')
plt.ylabel(r'$\sigma$ (Pa)')
plt.xscale('log')
#plt.legend()
#plt.savefig('0p08_prior.png',format='png')
plt.show()



In [ ]:
## Using NUTS algorithm to tune and draw samples ##
tune = 2000 #2000
draws = 10000 #50000 
with model:
    idata.extend(pm.sample(tune=tune, draws=draws, target_accept=0.85, random_seed=rng, discard_tuned_samples=True))
    #trace_pymc_ode = pm.sample(tune=tune, draws=draws)
print(idata)

In [ ]:
# Plotting trace plots #
az.plot_trace(idata,var_names=["mu_g","mu_eta_s","mu_yield_stress","mu_k","mu_n"],figsize=(12,20),rug=False,legend=True)

In [ ]:
az.plot_trace(idata,var_names=["sigma_g","sigma_eta_s","sigma_yield_stress","sigma_k","sigma_n"],figsize=(12,20),rug=False,legend=True)

In [ ]:
# Plotting trace plots #
az.plot_trace(idata,var_names=["z_G","z_eta_s","z_yield_stress","z_k","z_n"],figsize=(12,20),rug=False,legend=True)

In [ ]:
# Plotting autocorrelation plots #
az.plot_autocorr(idata,var_names=["mu_g","mu_eta_s","mu_yield_stress","mu_k","mu_n"])
#az.plot_autocorr(idata,var_names=["z_G","z_eta_s","z_yield_stress","z_k","z_n"])
#plt.savefig('Autocorr.png',format='png') #,bbox_inches='tight')


In [ ]:
# Plot to compare prior and posterior distribution of variables. Give appropriate variable name #
az.plot_dist_comparison(idata,var_names=["z_G"])

In [ ]:
# Summary of inference process #
trace = idata
az.summary(trace,var_names=["mu_g","mu_eta_s","mu_yield_stress","mu_k","mu_n","sigma_g","sigma_eta_s","sigma_yield_stress","sigma_k","sigma_n"])

In [ ]:
# Plotting data within imdata_posterior dataframe to understand data construction #
imdata_posterior = az.extract(idata, group = "posterior", num_samples=2).to_dataframe()
print(imdata_posterior)
cols = ["G", "eta_s", "yield_stress", "k", "n"]
row = imdata_posterior.iloc[1, :][cols].values
print(row)

In [ ]:
## Plotting predictions at shear rates used for training by sampling model parameters from posterior ##
## Note that second-level prior (NOT hyper-priors) are sampled and hence model parameters vary across shear rates used in training process ##

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"

num_samples = 500
imdata_posterior = az.extract(idata, group = "posterior", num_samples=num_samples).to_dataframe()
print(np.shape(imdata_posterior))
cols = ["G", "eta_s", "yield_stress", "k", "n","sigma"]

# arg: 0 (0.06/s); 1 (0.07/s); 2 (0.08/s); 3 (0.09/s); 4 (0.1/s);
arg = 0
shear_rate_ode = shear_rate_all[arg]
print(shear_rate_ode)
y0 = y0_all[arg]
# Using dimensional time and dimensional IC to get dimensional shear stress
stress_ode = odeint(dy_dt,y0,t,args=(G_true,eta_s_true,sigma_y_true,k_true,n_true,shear_rate_ode))
stress_analytical_pinn = shear_stress_analytical((G_true-G_min)/(G_max-G_min),(eta_s_true-eta_s_min)/(eta_s_max-eta_s_min)\
                                                 ,sigma_y_true/sigma_y_max,(k_true-k_min)/(k_max-k_min),(n_true-n_min)/(n_max-n_min),shear_rate_ode*np.ones(155),y0,times)
#shear_stress_expt = shear_stress_training[:,arg]

plt.plot(times,shear_stress_global[:,arg],'ok', label='Experiment Data')
plt.plot(times,stress_analytical_pinn,'^r', label='PINN analytical solution')
#plt.plot(times,stress_ode,'-b', label='PINN ODE solution')
# 0.06/s #
for row_idx in range(0,num_samples,1): # selects one sample every 5 samples to avoid correlation
  row = imdata_posterior.iloc[row_idx, :][cols].values
  theta = row
  # Using dimensional time as input to get dimensional shear_stress as output
  shear_stress_an = shear_stress_analytical(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode*np.ones(155),y0,times)
  #stress_odes = odeint(dy_dt,theta[5],t,args=(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode))
  # shear_stress_0p06s = shear_stress_an
  plt.plot(times,shear_stress_an,'-g',alpha=0.1)
plt.plot(np.NaN,np.NaN,'-g',alpha=0.1,label='Posterior Samples')

plt.title(r'$0.06$/s')
plt.xlabel(r'$t$ (s)')
plt.ylabel(r'$\sigma$ (Pa)')
plt.xscale('log')
plt.legend()
plt.grid()
plt.show()

arg = 1
shear_rate_ode = shear_rate_all[arg]
print(shear_rate_all)
print(shear_rate_ode)
y0 = y0_all[arg]
# Using dimensional time and dimensional IC to get dimensional shear stress
#stress_ode = odeint(dy_dt,y0,t,args=(G_true,eta_s_true,sigma_y_true,k_true,n_true,shear_rate_ode))
stress_analytical_pinn = shear_stress_analytical((G_true-G_min)/(G_max-G_min),(eta_s_true-eta_s_min)/(eta_s_max-eta_s_min)\
                                                 ,sigma_y_true/sigma_y_max,(k_true-k_min)/(k_max-k_min),(n_true-n_min)/(n_max-n_min),shear_rate_ode*np.ones(155),y0,times)
#shear_stress_expt = shear_stress_training[:,arg]

plt.plot(times,shear_stress_global[:,arg],'ok', label='Experiment Data')
plt.plot(times,stress_analytical_pinn,'^r', label='PINN analytical solution')
#plt.plot(times,stress_ode,'-b', label='PINN ODE solution')
for row_idx in range(num_samples,num_samples*2,1): # selects one sample every 5 samples to avoid correlation
  row = imdata_posterior.iloc[row_idx, :][cols].values
  theta = row
  # Using dimensional time as input to get dimensional shear_stress as output
  shear_stress_an = shear_stress_analytical(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode*np.ones(155),y0,times)
  #stress_odes = odeint(dy_dt,theta[5],t,args=(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode))
  # shear_stress_0p07s = shear_stress_an
  plt.plot(times,shear_stress_an,'-g',alpha=0.1)
plt.plot(np.NaN,np.NaN,'-g',alpha=0.1,label='Posterior Samples')

plt.title(r'$0.07$/s')
plt.xlabel(r'$t$ (s)')
plt.ylabel(r'$\sigma$ (Pa)')
plt.xscale('log')
plt.legend()
plt.grid()
plt.show()

arg = 2
shear_rate_ode = shear_rate_all[arg]
print(shear_rate_all)
print(shear_rate_ode)
y0 = y0_all[arg]
# Using dimensional time and dimensional IC to get dimensional shear stress
#stress_ode = odeint(dy_dt,y0,t,args=(G_true,eta_s_true,sigma_y_true,k_true,n_true,shear_rate_ode))
stress_analytical_pinn = shear_stress_analytical((G_true-G_min)/(G_max-G_min),(eta_s_true-eta_s_min)/(eta_s_max-eta_s_min)\
                                                 ,sigma_y_true/sigma_y_max,(k_true-k_min)/(k_max-k_min),(n_true-n_min)/(n_max-n_min),shear_rate_ode*np.ones(155),y0,times)
#shear_stress_expt = shear_stress_training[:,arg]

plt.plot(times,shear_stress_global[:,arg],'ok', label='Experiment Data')
plt.plot(times,stress_analytical_pinn,'^r', label='PINN analytical solution')
#plt.plot(times,stress_ode,'-b', label='PINN ODE solution')
for row_idx in range(num_samples*2,num_samples*3,1): # selects one sample every 5 samples to avoid correlation
  row = imdata_posterior.iloc[row_idx, :][cols].values
  theta = row
  # Using dimensional time as input to get dimensional shear_stress as output
  shear_stress_an = shear_stress_analytical(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode*np.ones(155),y0,times)
  #stress_odes = odeint(dy_dt,theta[5],t,args=(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode))
  # shear_stress_0p07s = shear_stress_an
  plt.plot(times,shear_stress_an,'-g',alpha=0.1)
plt.plot(np.NaN,np.NaN,'-g',alpha=0.1,label='Posterior Samples')

plt.title(r'$0.08$/s')
plt.xlabel(r'$t$ (s)')
plt.ylabel(r'$\sigma$ (Pa)')
plt.xscale('log')
plt.legend()
plt.grid()
plt.show()


In [ ]:
## Plotting predictions at all shear rates by sampling model parameters from posterior ##
## Note that first-level priors (or hyper-priors) are sampled and hence model parameters are shear-rate invariant ##
## Using hierarchical sampling based on sum and product rule ##
plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm" 

num_samples = 1000 # Hyper-prior samples 1000
thin_every = 40
itr_samples = num_samples*thin_every
no_samples = itr_samples*len(shear_rate_train) 
act_samples = no_samples/thin_every
act_samples = int(act_samples)
print(act_samples)
model_params_all = np.zeros((num_samples*act_samples,5)) 
noise_params = np.zeros((num_samples*act_samples,1)) 

# extracting normalized model parameters from posterior distribution #
imdata_pred = az.extract(idata, group = "posterior", num_samples=itr_samples).to_dataframe() #itr_samples
# Hyperpriors are same across all shear rates #
mean_cols = ["mu_g","mu_eta_s","mu_yield_stress","mu_k","mu_n"]
sigma_cols = ["sigma_g","sigma_eta_s","sigma_yield_stress","sigma_k","sigma_n"]
std_cols = ["z_G","z_eta_s","z_yield_stress","z_k","z_n"]
noise_cols = ["sigma"]

count = -1
for row_idx_1 in range(0,itr_samples,thin_every): 
    count = count + 1
    mean_row = imdata_pred.iloc[row_idx_1, :][mean_cols].values
    sigma_row = imdata_pred.iloc[row_idx_1, :][sigma_cols].values
    noise_row = imdata_pred.iloc[row_idx_1, :][noise_cols].values
    noise_dist = st.laplace(0,noise_row)
    count_1 = -1
    for row_idx_2 in range(0,no_samples,thin_every):
        count_1 = count_1 + 1
        
        z_row = imdata_pred.iloc[row_idx_2, :][std_cols].values
        G_predicted = mean_row[0]+sigma_row[0]*z_row[0]   #z_std.rvs(size=1)
        eta_s_predicted = mean_row[1]+sigma_row[1]*z_row[1]
        sigma_y_predicted = mean_row[2]+sigma_row[2]*z_row[2]
        k_predicted = mean_row[3]+sigma_row[3]*z_row[3]
        n_predicted = mean_row[4]+sigma_row[4]*z_row[4]
        
        model_params_all[count_1+(count)*act_samples,0] = G_predicted
        model_params_all[count_1+(count)*act_samples,1] = eta_s_predicted
        model_params_all[count_1+(count)*act_samples,2] = sigma_y_predicted
        model_params_all[count_1+(count)*act_samples,3] = k_predicted
        model_params_all[count_1+(count)*act_samples,4] = n_predicted
        #model_params_all[count_1+(count)*act_samples,5] = eta_s_predicted/G_predicted
        #model_params_all[count_1+(count)*act_samples,6] = k_predicted/G_predicted

        noise_params[count_1+(count)*act_samples,0] = noise_dist.rvs(size=1)
        
print(np.shape(model_params_all))
print(np.shape(noise_params))


In [ ]:
# Propogating uncertainty of model parameters into shear stress buildup profiles #
plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm" 

import matplotlib as mpl
fig = plt.figure(figsize=(16,9))
spec = mpl.gridspec.GridSpec(ncols=6,nrows=2)
ax1 = fig.add_subplot(spec[0,0:2])
ax2 = fig.add_subplot(spec[0,2:4])
ax3 = fig.add_subplot(spec[0,4:])
ax4 = fig.add_subplot(spec[1,1:3])
ax5 = fig.add_subplot(spec[1,3:5])

for arg in range(0,5): 
    shear_stress_all_prediction =  np.zeros((num_samples*act_samples,155))
    shear_rate_ode = shear_rate_all[arg]
    y0 = y0_all[arg]
    
    if arg==0:
        ax = ax1
        #plt.subplot(2,3,1)
    elif arg==1:
        ax = ax2
        #plt.subplot(2,3,2)
    elif arg==2:
        ax = ax3
        #plt.subplot(2,3,3)
    elif arg==3:
        ax = ax4
        #plt.subplot(2,3,4)
    elif arg==4:
        ax = ax5
        #plt.subplot(2,3,5)
    
    ax.plot(times,shear_stress_global[:,arg],'og', label='Experiment data')
    for row_idx_1 in range(0,num_samples): 
        for row_idx_2 in range(0,act_samples):
            theta = model_params_all[row_idx_2+row_idx_1*act_samples,:]
            shear_stress_an = shear_stress_analytical(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode*np.ones(155),y0,times)
            shear_stress_all_prediction[row_idx_2+row_idx_1*act_samples,:] = shear_stress_an + noise_params[row_idx_2+row_idx_1*act_samples,:]
            #plt.plot(times,shear_stress_an,'-y',alpha=0.5)
    #plt.plot(np.NaN,np.NaN,'-y',alpha=0.1,label='Posterior Samples')
    stress_low = np.zeros(np.size(times))
    stress_high = np.zeros(np.size(times))
    stress_mean = np.mean(shear_stress_all_prediction,axis=0)
    
    for itr in range(0,np.size(times)):
        stress_low[itr] = az.hdi(shear_stress_all_prediction[:,itr],0.95)[0]
        stress_high[itr] = az.hdi(shear_stress_all_prediction[:,itr],0.95)[1]
    stress_mean = np.mean(shear_stress_all_prediction,axis=0)
    ax.fill_between(times,stress_low,stress_high,color='b',alpha=0.2,label='95\% Credible interval')
        
    ax.set_title(r'$\dot\gamma_0 = %.2f~\rm{s^{-1}}$' % (shear_rate_ode))
    ax.set_xlabel(r'$t$ (s)')
    ax.set_ylabel(r'$\sigma$ (Pa)')
    ax.set_xscale('log')
    if arg==0:
        ax.legend()
   
plt.tight_layout()
plt.savefig('NEVP_shear_stress_subplots_0p06_0p07_0p08.eps',format='eps',bbox_inches='tight')
plt.savefig('NEVP_shear_stress_subplots_0p06_0p07_0p08.png',format='png',bbox_inches='tight')
plt.savefig(r'NEVP_shear_stress_subplots_0p06_0p07_0p08.pdf',format='pdf',bbox_inches='tight')
plt.show()



In [ ]:
## Plotting uncertainty of calibrated model parameters ##

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"
sns.set_style(None)

G_low = az.hdi(model_params_all[:,0],0.95)[0]*(G_max-G_min) + G_min
print(G_low)
G_high = az.hdi(model_params_all[:,0],0.95)[1]*(G_max-G_min) + G_min
print(G_high)

eta_s_low = az.hdi(model_params_all[:,1],0.95)[0]*(eta_s_max-eta_s_min) + eta_s_min
print(eta_s_low)
eta_s_high = az.hdi(model_params_all[:,1],0.95)[1]*(eta_s_max-eta_s_min) + eta_s_min
print(eta_s_high)

sigma_y_low = az.hdi(model_params_all[:,2],0.95)[0]*(sigma_y_max-sigma_y_min) + sigma_y_min 
print(sigma_y_low)
sigma_y_high = az.hdi(model_params_all[:,2],0.95)[1]*(sigma_y_max-sigma_y_min) + sigma_y_min
print(sigma_y_high)

k_low = az.hdi(model_params_all[:,3],0.95)[0]*(k_max-k_min) + k_min
print(k_low)
k_high = az.hdi(model_params_all[:,3],0.95)[1]*(k_max-k_min) + k_min
print(k_high)

n_low = az.hdi(model_params_all[:,4],0.95)[0]*(n_max-n_min) + n_min
print(n_low)
n_high = az.hdi(model_params_all[:,4],0.95)[1]*(n_max-n_min) + n_min
print(n_high)

import matplotlib as mpl
fig = plt.figure(figsize=(16,9))
spec = mpl.gridspec.GridSpec(ncols=6,nrows=2)
ax1 = fig.add_subplot(spec[0,0:2])
ax2 = fig.add_subplot(spec[0,2:4])
ax3 = fig.add_subplot(spec[0,4:])
ax4 = fig.add_subplot(spec[1,1:3])
ax5 = fig.add_subplot(spec[1,3:5])


ax1.plot([np.mean(model_params_all[:,0]*(G_max-G_min)+G_min,axis=0)],[5000.0],'o',color='k',markeredgewidth=4,label='Mean')
ax1.plot([G_low, G_high],[5000.0, 5000.0],'^',color='b',markeredgewidth=4, label='95\% Credible interval')
#ax.plot([np.mean(model_params_all[:,0])*(G_max-G_min)+G_min],[200.0],'s',color='r',markeredgewidth=4)
sns.histplot(model_params_all[:,0]*(G_max-G_min) + G_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax1)
#sns.despine(trim=True)
ax1.set_xlabel(r"$G~\rm{(Pa)}$")
ax1.ticklabel_format(axis='y',style='sci',scilimits=(0,0))
ax1.legend()

ax2.plot([np.mean(model_params_all[:,1]*(eta_s_max-eta_s_min)+eta_s_min,axis=0)],[5000.0],'o',color='k',markeredgewidth=4,label='Mean')
ax2.plot([eta_s_low, eta_s_high],[5000.0, 5000.0],'^',color='b',markeredgewidth=4)
#ax.plot([np.mean(model_params_all[:,1])*(eta_s_max-eta_s_min)+eta_s_min],[200.0],'s',color='r',markeredgewidth=4)
sns.histplot(model_params_all[:,1]*(eta_s_max-eta_s_min) + eta_s_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax2)
ax2.set_xlabel(r"$\eta_s~\rm{(Pa~s)}$")
ax2.ticklabel_format(axis='y',style='sci',scilimits=(0,0))


ax3.plot([np.mean(model_params_all[:,2]*(sigma_y_max-sigma_y_min)+sigma_y_min,axis=0)],[5000.0],'o',color='k',markeredgewidth=4,label='Mean')
ax3.plot([sigma_y_low, sigma_y_high],[5000.0, 5000.0],'^',color='b',markeredgewidth=4,label='95\% Credible interval')
#ax.plot([np.mean(model_params_all[:,2])*(sigma_y_max-sigma_y_min)+sigma_y_min],[200.0],'s',color='r',markeredgewidth=4,label='Prediction mean')
sns.histplot(model_params_all[:,2]*(sigma_y_max-sigma_y_min) + sigma_y_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax3)
ax3.set_xlabel(r"$\sigma_y~\rm{(Pa)}$")
ax3.ticklabel_format(axis='y',style='sci',scilimits=(0,0))

ax4.plot([np.mean(model_params_all[:,3]*(k_max-k_min)+k_min,axis=0)],[4000.0],'o',color='k',markeredgewidth=4,label='Mean')
ax4.plot([k_low, k_high],[4000.0, 4000.0],'^',color='b',markeredgewidth=4)
#ax.plot([np.mean(model_params_all[:,3])*(k_max-k_min)+k_min],[200.0],'s',color='r',markeredgewidth=4)
sns.histplot(model_params_all[:,3]*(k_max-k_min) + k_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax4)
ax4.set_xlabel(r"$K~\rm{(Pa~s^n)}$")
ax4.ticklabel_format(axis='y',style='sci',scilimits=(0,0))

ax5.plot([np.mean(model_params_all[:,4]*(n_max-n_min)+n_min,axis=0)],[5000.0],'o',color='k',markeredgewidth=4,label='Mean')
ax5.plot([n_low, n_high],[5000.0, 5000.0],'^',color='b',markeredgewidth=4)
#ax.plot([np.mean(model_params_all[:,4])*(n_max-n_min)+n_min],[200.0],'s',color='r',markeredgewidth=4)
sns.histplot(model_params_all[:,4]*(n_max-n_min) + n_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax5)
ax5.set_xlabel(r"$n~\rm{(-)}$")
ax5.ticklabel_format(axis='y',style='sci',scilimits=(0,0))

plt.tight_layout()
plt.savefig('NEVP_model_params_subplots_0p06_0p07_0p08.eps',format='eps',bbox_inches='tight')
plt.savefig('NEVP_model_params_subplots_0p06_0p07_0p08.png',format='png',bbox_inches='tight')
plt.savefig(r'NEVP_model_params_subplots_0p06_0p07_0p08.pdf',format='pdf',bbox_inches='tight')
plt.show()

In [ ]:
## Corner plot of sampled data ##

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm" 

xvars = [r"$G~\rm{(Pa)}$",r"$\eta_s~\rm{(Pa s)}$",r"$\sigma_y~\rm{(Pa)}$",r"$K~\rm{(Pa s^n)}$",r"$n$"]
#model_params_all_sub = model_params_all[1:10000000,:]
pairplotdata = pd.DataFrame(data=model_params_all,columns=["G", "eta_s", "Yield_stress", "k", "n"])
pairplotdata["G"] = pairplotdata["G"]*(G_max-G_min) + G_min
pairplotdata["eta_s"] = pairplotdata["eta_s"]*(eta_s_max-eta_s_min) + eta_s_min
pairplotdata["Yield_stress"] = pairplotdata["Yield_stress"]*(sigma_y_max-sigma_y_min) + sigma_y_min
pairplotdata["k"] = pairplotdata["k"]*(k_max-k_min) + k_min
pairplotdata["n"] = pairplotdata["n"]
pp1 = sns.pairplot(pairplotdata,vars=["G", "eta_s", "Yield_stress", "k", "n"],kind="scatter")
pp1.x_vars = xvars
pp1.y_vars = xvars
pp1._add_axis_labels()


plt.savefig('NEVP_Pairplot.png',format='png',bbox_inches='tight')
#plt.savefig('NEVP_Pairplot.eps',format='eps',bbox_inches='tight')
plt.savefig('NEVP_Pairplot.pdf',format='pdf',bbox_inches='tight')
plt.show()

In [ ]:
## Q-Q plot ##

shear_stress_all_prediction =  np.zeros((200,155)) # Holding data for 2 different shear rates (100 for each): test data
for itr in range(0,200,2):
    rand_list = [*range(1,num_samples*act_samples),1]
    #print(rand_list)
    model_param_idx = random.choice(rand_list)
    # print(model_param_idx)
    theta = model_params_all[model_param_idx,:]
    for arg in range(3,5):
        shear_rate_ode = shear_rate_all[arg]
        y0 = y0_all[arg]
        shear_stress_an = shear_stress_analytical(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode,y0,times)
        if arg == 3:
            shear_stress_all_prediction[itr,:] = shear_stress_an + noise_params[model_param_idx,:]
        elif arg == 4:
            shear_stress_all_prediction[itr+1,:] = shear_stress_an + noise_params[model_param_idx,:]

shear_stress_predict = shear_stress_all_prediction.reshape(-1,1)
print(np.shape(shear_stress_predict))  

shear_stress_test = np.zeros(310,)
shear_stress_test[0:155] = shear_stress_global[:,3]
shear_stress_test[155:310] = shear_stress_global[:,4]
## Q-Q plot ##

def qqplot(x, y):
    """Draw a quantile-quantile plot for x vs y`.
 
    Parameters
    ----------
    x, y : array-like
        One-dimensional numeric arrays.
 
    ax : matplotlib.axes.Axes, optional
        Axes on which to plot. If not provided, the current axes will be used.
 
    quantiles : int or array-like, number of evenly spaced quantiles
    """
    yy = np.linspace(10,45)
    quantiles = 20 #min(len(x), len(y))
 
    # Compute quantiles of the two samples
    if isinstance(quantiles, numbers.Integral):
        quantiles = np.linspace(start=0, stop=1, num=int(quantiles))
    else:
        quantiles = np.atleast_1d(np.sort(quantiles))
    x_quantiles = np.quantile(x, quantiles, method="nearest")
    y_quantiles = np.quantile(y, quantiles, method="nearest")
    plt.plot(yy,yy,'-k',label='abscissa=ordinate line')
    plt.scatter(x_quantiles, y_quantiles, c='r', label='Quantiles')
    plt.xlim([25,45])
    plt.ylim([25,45])
    plt.xlabel(r'$\sigma$ (Pa)')
    plt.ylabel(r'$\sigma_\mathrm{exp}$ (Pa)')
    plt.legend()
    plt.gca().set_aspect('equal')
    plt.savefig('NEVP_qq_plot.pdf',format='pdf',bbox_inches='tight')
    plt.savefig('NEVP_qq_plot.png',format='png',bbox_inches='tight')
    plt.savefig('NEVP_qq_plot.eps',format='eps',bbox_inches='tight')
    plt.show()

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"    
# shear_stress_predict contains shear stress obtained from last set of sampled unknown model parameters
qqplot(shear_stress_predict,shear_stress_test)


In [ ]:
## Posterior Predictive Checks ##

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm" 

with model:
 pm.sample_posterior_predictive(idata, extend_inferencedata=True, random_seed=rng) #imdata

#print(idata)
#labels = ['Posterior predictive', r'$\sigma_\mathrm{exp}$ (Pa)', 'Posterior predictive mean']
az.plot_ppc(idata, num_pp_samples=100, colors = ["b","g","k"])
plt.legend(fontsize="14")
plt.xlabel(r"$\sigma$ (Pa)")
plt.ylabel("Likelihood")
plt.savefig('NEVP_ppc.pdf',format='pdf') 
plt.savefig('NEVP_ppc.png',format='png') 
plt.savefig('NEVP_ppc.eps',format='eps') 

In [ ]:
## Saving thermal resistance data ##

R_total_selected = R_total[0:1000000]
R_bulk_selected = R_bulk[0:1000000]
R_contact_selected = R_c[0:1000000]
R_NEVP = np.zeros((1000000,3))
R_NEVP[:,0] = R_total_selected
R_NEVP[:,1] = R_bulk_selected
R_NEVP[:,2] = R_contact_selected
print(np.shape(R_NEVP))


resistance_NEVP_df = pd.DataFrame(R_NEVP,columns=["Total","Bulk","Contact"])
print(np.shape(resistance_NEVP_df))
resistance_NEVP_df.to_excel("R_total_NEVP.xlsx")

## Training data: 0.08, 0.09 and 0.1 /s ##

In [ ]:
## Data processing ##

y0_train = [11.966,12.478,14.668]
shear_rate_train = [0.08,0.09,0.1]

shear_stress_training = np.zeros(465,)
shear_stress_training[0:155] = shear_stress_global[:,2]
shear_stress_training[155:310] = shear_stress_global[:,3]
shear_stress_training[310:465] = shear_stress_global[:,4]
print(shear_stress_training) # contains data for 0.08/s, 0.09/s and 0.1/s
print(np.shape(shear_stress_training)) 

# Appending time to create times_1d #
times_1d = np.append(times,times)
times_1d = np.append(times_1d,times)
times_1d = times_1d.reshape(-1)
print(np.shape(times_1d))

# Shear rate index #
shear_rate_0p08_idx = 0*np.ones(155)
shear_rate_0p09_idx = 1*np.ones(155)
shear_rate_0p1_idx = 2*np.ones(155)
shear_rate_idx = np.append(shear_rate_0p08_idx,shear_rate_0p09_idx)
shear_rate_idx = np.append(shear_rate_idx,shear_rate_0p1_idx)
shear_rate_idx = shear_rate_idx.reshape(-1)
shear_rate_idx = np.asarray(shear_rate_idx,dtype='int')
print(np.shape(shear_rate_idx))

# shear rates under consideration #
shear_rate_0p08 = 0.08*np.ones(155)
shear_rate_0p09 = 0.09*np.ones(155)
shear_rate_0p1 = 0.1*np.ones(155)
shear_rate_1d = np.append(shear_rate_0p08,shear_rate_0p09) 
shear_rate_1d = np.append(shear_rate_1d,shear_rate_0p1) 
shear_rate_1d = shear_rate_1d.reshape(-1)
print(np.shape(shear_rate_1d))

# ICs under consideration #
stress_ic_0p08 = y0_all[2]*np.ones(155)
stress_ic_0p09 = y0_all[3]*np.ones(155)
stress_ic_0p1 = y0_all[4]*np.ones(155)
stress_ic_1d = np.append(stress_ic_0p08,stress_ic_0p09) 
stress_ic_1d = np.append(stress_ic_1d,stress_ic_0p1) 
stress_ic_1d = stress_ic_1d.reshape(-1)
print(np.shape(stress_ic_1d))

In [ ]:
## To define dimensionality of unknown model parameters in the model definition below ##
coords = {"shear_rate_coord": shear_rate_train}

In [ ]:
## Defining the UQ model ##
with pm.Model(coords=coords) as model_0p08_0p09_0p1:
    time_data = pm.ConstantData("time_data", times_1d, dims="obs_id")
    print(pm.draw(time_data).size)
    
    # Hyper-priors: inference space between 0 and 1 #
    #mu_g = pm.LogNormal("mu_g",mu=0.0,sigma=1.0) #0.0 0.25
    mu_g = pm.Beta("mu_g",alpha=2.0,beta=2.0) 
    sigma_g = pm.HalfNormal("sigma_g",sigma=0.05) #0.01
    
    #mu_eta_s = pm.LogNormal("mu_eta_s",mu=0.0,sigma=1.0) 
    mu_eta_s = pm.Beta("mu_eta_s",alpha=2.0,beta=2.0)
    sigma_eta_s = pm.HalfNormal("sigma_eta_s",sigma=0.05)
    
    #mu_yield_stress = pm.HalfNormal("mu_yield_stress",sigma=0.1) 
    mu_yield_stress = pm.Beta("mu_yield_stress",alpha=2.0,beta=2.0)
    sigma_yield_stress = pm.HalfNormal("sigma_yield_stress",sigma=0.05)
    
    #mu_k = pm.HalfNormal("mu_k",sigma=0.5) #0.5
    mu_k = pm.Beta("mu_k",alpha=2.0,beta=2.0)
    sigma_k = pm.HalfNormal("sigma_k",sigma=0.05) # 0.05 increasing this sigma value leads to unphysical stress values - 1e32 or so
    
    mu_n = pm.Beta("mu_n",alpha=2.0,beta=2.0) # Beta because n = [0,1] 
    sigma_n = pm.HalfNormal("sigma_n",sigma=0.05)
    
    # Priors: normalized and dimensional unknown model parameters using non-centered paramterization #
    z_G = pm.Normal("z_G",mu=0,sigma=1,dims="shear_rate_coord")
    G = pm.Deterministic("G", mu_g+sigma_g*z_G, dims="shear_rate_coord")
    G_dd = pm.Deterministic("G_dd", G*(G_max-G_min)+G_min, dims="shear_rate_coord")
    
    z_eta_s = pm.Normal("z_eta_s",mu=0,sigma=1,dims="shear_rate_coord")
    eta_s = pm.Deterministic("eta_s", mu_eta_s + sigma_eta_s*z_eta_s, dims="shear_rate_coord")
    eta_s_dd = pm.Deterministic("eta_s_dd", eta_s*(eta_s_max-eta_s_min)+eta_s_min, dims="shear_rate_coord")
    
    z_yield_stress = pm.Normal("z_yield_stress",mu=0,sigma=1,dims="shear_rate_coord")
    yield_stress = pm.Deterministic("yield_stress", mu_yield_stress + sigma_yield_stress*z_yield_stress , dims="shear_rate_coord")
    yield_stress_dd = pm.Deterministic("yield_stress_dd", yield_stress*(sigma_y_max-sigma_y_min)+sigma_y_min, dims="shear_rate_coord")

    z_k = pm.Normal("z_k",mu=0,sigma=1,dims="shear_rate_coord")
    k = pm.Deterministic("k", mu_k + sigma_k*z_k, dims="shear_rate_coord")
    k_dd = pm.Deterministic("k_dd", k*(k_max-k_min)+k_min, dims="shear_rate_coord")

    z_n = pm.Normal("z_n",mu=0,sigma=1,dims="shear_rate_coord")
    n = pm.Deterministic("n", mu_n + sigma_n*z_n, dims="shear_rate_coord")
    n_dd = pm.Deterministic("n_dd", n*(n_max-n_min)+n_min, dims="shear_rate_coord")

    # Experiment measurement uncertainty #
    sigma = pm.Exponential("sigma", 2.0) # rate parameter = 5.0  2.0
    # sigma = pm.HalfNormal("sigma",sigma=2.0) # rate parameter = 5.0  2.0
    
    # Likelihood mean based on analytical solution #
    shear_stress_mean = (yield_stress_dd[shear_rate_idx] + k_dd[shear_rate_idx]*shear_rate_1d**n_dd[shear_rate_idx])* \
    (1 - (1-(stress_ic_1d/(yield_stress_dd[shear_rate_idx] + k_dd[shear_rate_idx]*shear_rate_1d**n_dd[shear_rate_idx]))) * \
    np.exp(-G_dd[shear_rate_idx]*shear_rate_1d*(time_data-0.031)/(yield_stress_dd[shear_rate_idx]+ k_dd[shear_rate_idx]*shear_rate_1d**n_dd[shear_rate_idx] + eta_s_dd[shear_rate_idx]*shear_rate_1d)))
    
    print(pm.draw(shear_stress_mean).shape)

    # Likelihood #
    pm.Laplace("Y_obs", mu=shear_stress_mean, b=sigma, observed=shear_stress_training, dims = "obs_id") 
    idata_0p08_0p09_0p1 = pm.sample_prior_predictive(samples=10000, random_seed=rng)

print(idata_0p08_0p09_0p1)

In [ ]:
# sampler = "NUTS PyMC ODE"
## Using NUTS algorithm to tune and draw samples ##
tune = 2000 #2000
draws = 10000 #50000 
with model_0p08_0p09_0p1:
    idata_0p08_0p09_0p1.extend(pm.sample(tune=tune, draws=draws, target_accept=0.85, random_seed=rng, discard_tuned_samples=True))
    #trace_pymc_ode = pm.sample(tune=tune, draws=draws)
print(idata_0p08_0p09_0p1)

In [ ]:
## Plotting predictions at all shear rates by sampling model parameters from posterior ##
## Note that first-level priors (or hyper-priors) are sampled and hence model parameters are shear-rate invariant ##
## Using hierarchical sampling based on sum and product rule ##
plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm" 

num_samples = 1000 
thin_every = 40
itr_samples = num_samples*thin_every
no_samples = itr_samples*len(shear_rate_train) 
act_samples = no_samples/thin_every
act_samples = int(act_samples)
print(act_samples)
model_params_all_2 = np.zeros((num_samples*act_samples,5)) 
noise_params_2 = np.zeros((num_samples*act_samples,1)) 

# extracting normalized model parameters from posterior distribution #
imdata_pred_0p08_0p09_0p1 = az.extract(idata_0p08_0p09_0p1, group = "posterior", num_samples=itr_samples).to_dataframe() 
# Hyperpriors are same across all shear rates #
mean_cols = ["mu_g","mu_eta_s","mu_yield_stress","mu_k","mu_n"]
sigma_cols = ["sigma_g","sigma_eta_s","sigma_yield_stress","sigma_k","sigma_n"]
std_cols = ["z_G","z_eta_s","z_yield_stress","z_k","z_n"]
noise_cols = ["sigma"]

count = -1
for row_idx_1 in range(0,itr_samples,thin_every): 
    count = count + 1
    mean_row = imdata_pred_0p08_0p09_0p1.iloc[row_idx_1, :][mean_cols].values
    sigma_row = imdata_pred_0p08_0p09_0p1.iloc[row_idx_1, :][sigma_cols].values
    noise_row = imdata_pred_0p08_0p09_0p1.iloc[row_idx_1, :][noise_cols].values
    noise_dist = st.laplace(0,noise_row)
    count_1 = -1
    for row_idx_2 in range(0,no_samples,thin_every):
        count_1 = count_1 + 1
        print(count_1)
        
        z_row = imdata_pred_0p08_0p09_0p1.iloc[row_idx_2, :][std_cols].values
        G_predicted = mean_row[0]+sigma_row[0]*z_row[0]   #z_std.rvs(size=1)
        eta_s_predicted = mean_row[1]+sigma_row[1]*z_row[1]
        sigma_y_predicted = mean_row[2]+sigma_row[2]*z_row[2]
        k_predicted = mean_row[3]+sigma_row[3]*z_row[3]
        n_predicted = mean_row[4]+sigma_row[4]*z_row[4]
        
        model_params_all_2[count_1+(count)*act_samples,0] = G_predicted
        model_params_all_2[count_1+(count)*act_samples,1] = eta_s_predicted
        model_params_all_2[count_1+(count)*act_samples,2] = sigma_y_predicted
        model_params_all_2[count_1+(count)*act_samples,3] = k_predicted
        model_params_all_2[count_1+(count)*act_samples,4] = n_predicted

        noise_params_2[count_1+(count)*act_samples,0] = noise_dist.rvs(size=1)
        
print(np.shape(model_params_all_2))
print(np.shape(noise_params_2))


In [ ]:
## Plotting posterior distributions at all shear-rates (training and test) under consideration ##

fig = plt.figure(figsize=(16,9))
spec = mpl.gridspec.GridSpec(ncols=6,nrows=2)
ax1 = fig.add_subplot(spec[0,0:2])
ax2 = fig.add_subplot(spec[0,2:4])
ax3 = fig.add_subplot(spec[0,4:])
ax4 = fig.add_subplot(spec[1,1:3])
ax5 = fig.add_subplot(spec[1,3:5])

for arg in range(0,5):

    shear_stress_all_prediction = np.zeros((num_samples*act_samples,155))
    shear_stress_all_prediction_2 = np.zeros((num_samples*act_samples,155))
    shear_rate_ode = shear_rate_all[arg]
    y0 = y0_all[arg]
    
    if arg==0:
        ax = ax1
        #plt.subplot(2,3,1)
    elif arg==1:
        ax = ax2
        #plt.subplot(2,3,2)
    elif arg==2:
        ax = ax3
        #plt.subplot(2,3,3)
    elif arg==3:
        ax = ax4
        #plt.subplot(2,3,4)
    elif arg==4:
        ax = ax5
        #plt.subplot(2,3,5)
    
    ax.plot(times,shear_stress_global[:,arg],'og', label='Experiment data')

    for row_idx_1 in range(0,num_samples): 
        for row_idx_2 in range(0,act_samples):
            theta = model_params_all[row_idx_2+row_idx_1*act_samples,:]
            shear_stress_an = shear_stress_analytical(theta[0],theta[1],theta[2],theta[3],theta[4],shear_rate_ode*np.ones(155),y0,times)
            shear_stress_all_prediction[row_idx_2+row_idx_1*act_samples,:] = shear_stress_an + noise_params[row_idx_2+row_idx_1*act_samples,0]

            theta_2 = model_params_all_2[row_idx_2+row_idx_1*act_samples,:]
            shear_stress_an_2 = shear_stress_analytical(theta_2[0],theta_2[1],theta_2[2],theta_2[3],theta_2[4],shear_rate_ode*np.ones(155),y0,times)
            shear_stress_all_prediction_2[row_idx_2+row_idx_1*act_samples,:] = shear_stress_an_2 + noise_params_2[row_idx_2+row_idx_1*act_samples,0]
       
    stress_low = np.zeros(np.size(times))
    stress_high = np.zeros(np.size(times))
   

    stress_low_2 = np.zeros(np.size(times))
    stress_high_2 = np.zeros(np.size(times))
        
    for itr in range(0,np.size(times)):
        stress_low[itr] = az.hdi(shear_stress_all_prediction[:,itr],0.95)[0]
        stress_high[itr] = az.hdi(shear_stress_all_prediction[:,itr],0.95)[1]

        stress_low_2[itr] = az.hdi(shear_stress_all_prediction_2[:,itr],0.95)[0]
        stress_high_2[itr] = az.hdi(shear_stress_all_prediction_2[:,itr],0.95)[1]
        
    ax.fill_between(times,stress_low_2,stress_high_2,color='r',alpha=0.2,label=r'$\dot\gamma_{0} = 0.08~\rm{s^{-1}}, 0.09~\rm{s^{-1}}, 0.10~\rm{s^{-1}}$')
    ax.fill_between(times,stress_low,stress_high,color='b',alpha=0.2,label=r'$\dot\gamma_{0} = 0.06~\rm{s^{-1}}, 0.07~\rm{s^{-1}}, 0.08~\rm{s^{-1}}$')
    
    ax.set_title(r'$\dot\gamma_0 = %.2f~\rm{s^{-1}}$' % shear_rate_ode)
    ax.set_xlabel(r'$t$ (s)')
    ax.set_ylabel(r'$\sigma$ (Pa)')
    ax.set_xscale('log')
    if arg==0:
        ax.legend()
    # plt.savefig(r'NNEVP_%.2fs_0p08_0p09_0p1.png' % (shear_rate_ode),format='png',bbox_inches='tight')
    # plt.savefig(r'NNEVP_%.2fs_0p08_0p09_0p1.eps' % (shear_rate_ode),format='eps',bbox_inches='tight')
plt.tight_layout()
plt.savefig('NEVP_shear_stress_subplots_0p8_0p9_0p10.eps',format='eps',bbox_inches='tight')
plt.savefig('NEVP_shear_stress_subplots_0p8_0p9_0p10.png',format='png',bbox_inches='tight')
plt.savefig(r'NEVP_shear_stress_subplots_0p8_0p9_0p10.pdf',format='pdf',bbox_inches='tight')
plt.show()


In [ ]:
# Comparing model parameters distribution  #

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 16})
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"
sns.set_style(None)

fig = plt.figure(figsize=(16,9))
spec = mpl.gridspec.GridSpec(ncols=6,nrows=2)
ax1 = fig.add_subplot(spec[0,0:2])
ax2 = fig.add_subplot(spec[0,2:4])
ax3 = fig.add_subplot(spec[0,4:])
ax4 = fig.add_subplot(spec[1,1:3])
ax5 = fig.add_subplot(spec[1,3:5])

G_low = az.hdi(model_params_all_2[:,0],0.95)[0]*(G_max-G_min) + G_min
G_high = az.hdi(model_params_all_2[:,0],0.95)[1]*(G_max-G_min) + G_min

eta_s_low = az.hdi(model_params_all_2[:,1],0.95)[0]*(eta_s_max-eta_s_min) + eta_s_min
eta_s_high = az.hdi(model_params_all_2[:,1],0.95)[1]*(eta_s_max-eta_s_min) + eta_s_min

sigma_y_low = az.hdi(model_params_all_2[:,2],0.95)[0]*(sigma_y_max-sigma_y_min) + sigma_y_min 
sigma_y_high = az.hdi(model_params_all_2[:,2],0.95)[1]*(sigma_y_max-sigma_y_min) + sigma_y_min

k_low = az.hdi(model_params_all_2[:,3],0.95)[0]*(k_max-k_min) + k_min
k_high = az.hdi(model_params_all_2[:,3],0.95)[1]*(k_max-k_min) + k_min

n_low = az.hdi(model_params_all_2[:,4],0.95)[0]*(n_max-n_min) + n_min
n_high = az.hdi(model_params_all_2[:,4],0.95)[1]*(n_max-n_min) + n_min

sns.histplot(model_params_all[:,0]*(G_max-G_min) + G_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax1,label=r'$\dot\gamma_{0} = 0.06~\rm{s^{-1}}, 0.07~\rm{s^{-1}}, 0.08~\rm{s^{-1}}$')
sns.histplot(model_params_all_2[:,0]*(G_max-G_min) + G_min,fill=True,color='r',bins=50,alpha=0.2,ax=ax1,label=r'$\dot\gamma_{0} = 0.08~\rm{s^{-1}}, 0.09~\rm{s^{-1}}, 0.10~\rm{s^{-1}}$')

ax1.set_xlabel(r"$G~\rm{(Pa)}$")
ax1.ticklabel_format(axis='y',style='sci',scilimits=(0,0))
ax1.legend()

sns.histplot(model_params_all[:,1]*(eta_s_max-eta_s_min) + eta_s_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax2)
sns.histplot(model_params_all_2[:,1]*(eta_s_max-eta_s_min) + eta_s_min,fill=True,color='r',bins=50,alpha=0.2,ax=ax2)
ax2.set_xlabel(r"$\eta_s~\rm{(Pa~s)}$")
ax2.ticklabel_format(axis='y',style='sci',scilimits=(0,0))

sns.histplot(model_params_all[:,2]*(sigma_y_max-sigma_y_min) + sigma_y_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax3)
sns.histplot(model_params_all_2[:,2]*(sigma_y_max-sigma_y_min) + sigma_y_min,fill=True,color='r',bins=50,alpha=0.2,ax=ax3)
ax3.set_xlabel(r"$\sigma_y~\rm{(Pa)}$")
ax3.ticklabel_format(axis='y',style='sci',scilimits=(0,0))

sns.histplot(model_params_all[:,3]*(k_max-k_min) + k_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax4)
sns.histplot(model_params_all_2[:,3]*(k_max-k_min) + k_min,fill=True,color='r',bins=50,alpha=0.2,ax=ax4)
ax4.set_xlabel(r"$K~\rm{(Pa~s^n)}$")
ax4.ticklabel_format(axis='y',style='sci',scilimits=(0,0))

sns.histplot(model_params_all[:,4]*(n_max-n_min) + n_min,fill=True,color='b',bins=50,alpha=0.2,ax=ax5)
sns.histplot(model_params_all_2[:,4]*(n_max-n_min) + n_min,fill=True,color='r',bins=50,alpha=0.2,ax=ax5)
ax5.set_xlabel(r"$n~\rm{(-)}$")
ax5.ticklabel_format(axis='y',style='sci',scilimits=(0,0))

plt.tight_layout()
plt.savefig('NEVP_model_params_subplots_0p08_0p09_0p10.eps',format='eps',bbox_inches='tight')
plt.savefig('NEVP_model_params_subplots_0p08_0p09_0p10.png',format='png',bbox_inches='tight')
plt.savefig(r'NEVP_model_params_subplots_0p08_0p09_0p10.pdf',format='pdf',bbox_inches='tight')
plt.show()

